In [1]:
import sys
sys.path.append(r'../../')

In [2]:
from tfcl.models.ptavit3dca.ptavit3dca_dn import *

In [3]:
NClasses=2 #                                                       
nf=96                                                              
verbose = True  # print only on global_rank==0     
model_config = {'in_channels_s2':4,
                'in_channels_s1':5,
               'spatial_size_init':(128,128),                      
               'depths':[2,2,5,2],                                 
               'nfilters_init':nf,                                 
               'nheads_start':nf//4,                               
               'NClasses':NClasses,                                
               'verbose':verbose,                                  
               'segm_act':'sigmoid'}                               
                                                                   
# UNet-like model                                                  
model = ptavit3dca_dn(**model_config) 


 Using stem normalization 
 @@@@@@@@@@@@@ Going DOWN @@@@@@@@@@@@@@@@@@@ 
depth:= 0, layer_dim_in: 96, layer_dim: 96, stage_depth::2, spatial_size::(32, 32), scales::[16, 8, 8]
depth:= 1, layer_dim_in: 96, layer_dim: 192, stage_depth::2, spatial_size::(16, 16), scales::[32, 4, 4]
depth:= 2, layer_dim_in: 192, layer_dim: 384, stage_depth::5, spatial_size::(8, 8), scales::[64, 2, 2]
depth:= 3, layer_dim_in: 384, layer_dim: 768, stage_depth::2, spatial_size::(4, 4), scales::[128, 1, 1]
 XXXXXXXXXXXXXXXXXXXXX Coming up XXXXXXXXXXXXXXXXXXXXXXXXX 
depth:= 4, layer_dim_in: 384, layer_dim: 384, stage_depth::5, spatial_size::(8, 8), scales::[64, 2, 2]
depth:= 5, layer_dim_in: 192, layer_dim: 192, stage_depth::2, spatial_size::(16, 16), scales::[32, 4, 4]
depth:= 6, layer_dim_in: 96, layer_dim: 96, stage_depth::2, spatial_size::(32, 32), scales::[16, 8, 8]


In [4]:
# Depending on Hardware, reduce spatial size to run on demo. Tested on cpu with 32GB of memory
# Random 3D input batch size of 2, 5 s1 channels, 4 s2 channels, 4 time instances, height x width = 64 x 64
b,c1,c2,t,h,w = 2,5,4,4,128,128
ins1 = torch.rand(b,c1,t,h,w)
ins2 = torch.rand(b,c2,t,h,w)

In [5]:
print("input shape S2::{}, shape S1::{}".format(ins2.shape,ins1.shape))
outs = model(ins2,ins1)

input shape S2::torch.Size([2, 4, 4, 128, 128]), shape S1::torch.Size([2, 5, 4, 128, 128])


In [6]:
outs.shape

torch.Size([2, 6, 128, 128])

In [7]:
preds_e = outs[:,:NClasses]
preds_b = outs[:,NClasses:2*NClasses]
preds_d = outs[:,2*NClasses:3*NClasses]

In [8]:
print("extent shape::{}".format(preds_e.shape))
print("bounds shape::{}".format(preds_b.shape))
print("distance shape::{}".format(preds_d.shape))

extent shape::torch.Size([2, 2, 128, 128])
bounds shape::torch.Size([2, 2, 128, 128])
distance shape::torch.Size([2, 2, 128, 128])
